In [ ]:
path = 'crawl-data/CC-MAIN-2022-40/segments/1664030331677.90/warc/CC-MAIN-20220924151538-20220924181538-00000.warc.gz'
prefix = ' s3:///

In [14]:
#!/usr/bin/env python3

import boto3

SOURCE_PROFILE="tipster"
SOURCE_BUCKET="commoncrawl"
SOURCE_PREFIX="crawl-data/CC-NEWS/2022/02/" #.warc.gz"

DEST_PROFILE="tipster"
DEST_BUCKET="emr-job-source-bkt2"
DEST_PREFIX="output"

todo = {}

# Get a list of all items in the source bucket
s3_source = boto3.Session(profile_name=SOURCE_PROFILE).client('s3')
for page in s3_source.get_paginator('list_objects_v2').paginate(Bucket=SOURCE_BUCKET, Prefix=SOURCE_PREFIX):
    for cur in page.get('Contents', []):
        key = cur['Key'][len(SOURCE_PREFIX):] 
        todo[key] = cur['Size']

# # And now remove any item in the destionation that's the correct size
s3_dest = boto3.Session(profile_name=DEST_PROFILE).client('s3')
for page in s3_dest.get_paginator('list_objects_v2').paginate(Bucket=DEST_BUCKET, Prefix=DEST_PREFIX):
    for cur in page.get('Contents', []):
        key = cur['Key'][len(DEST_PREFIX):]
        if key in todo and todo[key] == cur['Size']:
            del todo[key]

# Copy all of the items
for cur, size in todo.items():
    print("Copy " + cur)
    if size <= 8388608:
        # A simple copy
        resp = s3_source.get_object(Bucket=SOURCE_BUCKET, Key=SOURCE_PREFIX + cur)
        s3_dest.put_object(Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur, Body=resp['Body'].read())
    else:
        print('Selecting Multi-Part Upload')
        # Use a multipart upload
        mp = s3_dest.create_multipart_upload(Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur)
        parts = []
        for part, offset in enumerate(range(0, size, 8388608)):
            resp = s3_source.get_object(Bucket=SOURCE_BUCKET, Key=SOURCE_PREFIX + cur, Range=f'bytes={offset}-{offset+8388607}')
            temp = s3_dest.upload_part(
                Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur,
                Body=resp['Body'].read(),
                PartNumber=part+1,
                UploadId=mp['UploadId'],
            )
            parts.append({'PartNumber': part+1, 'ETag': temp['ETag']})
        s3_dest.complete_multipart_upload(
            Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur,
            UploadId=mp['UploadId'],
            MultipartUpload={'Parts': parts}
        )
    break

Copy CC-NEWS-20220201010205-00084.warc.gz
Selecting Multi-Part Upload


In [ ]:
#!/usr/bin/env python3

import boto3

SOURCE_PROFILE="tipster"
SOURCE_BUCKET="commoncrawl"
SOURCE_PREFIX="crawl-data/CC-NEWS/2022/02/" #.warc.gz"

DEST_PROFILE="tipster"
DEST_BUCKET="emr-job-source-bkt2"
DEST_PREFIX="output"

todo = {}

# Get a list of all items in the source bucket
s3_source = boto3.Session(profile_name=SOURCE_PROFILE).client('s3')
for page in s3_source.get_paginator('list_objects_v2').paginate(Bucket=SOURCE_BUCKET, Prefix=SOURCE_PREFIX):
    for cur in page.get('Contents', []):
        key = cur['Key'][len(SOURCE_PREFIX):] 
        todo[key] = cur['Size']

# # And now remove any item in the destionation that's the correct size
s3_dest = boto3.Session(profile_name=DEST_PROFILE).client('s3')
for page in s3_dest.get_paginator('list_objects_v2').paginate(Bucket=DEST_BUCKET, Prefix=DEST_PREFIX):
    for cur in page.get('Contents', []):
        key = cur['Key'][len(DEST_PREFIX):]
        if key in todo and todo[key] == cur['Size']:
            del todo[key]

            
# Copy all of the items
for cur, size in todo.items():
    print("Copy " + cur)
    if size <= 8388608:
        # A simple copy
        resp = s3_source.get_object(Bucket=SOURCE_BUCKET, Key=SOURCE_PREFIX + cur)
        s3_dest.put_object(Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur, Body=resp['Body'].read())
    else:
        print('Selecting Multi-Part Upload')
        # Use a multipart upload
        mp = s3_dest.create_multipart_upload(Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur)
        parts = []
        for part, offset in enumerate(range(0, size, 8388608)):
            resp = s3_source.get_object(Bucket=SOURCE_BUCKET, Key=SOURCE_PREFIX + cur, Range=f'bytes={offset}-{offset+8388607}')
            temp = s3_dest.upload_part(
                Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur,
                Body=resp['Body'].read(),
                PartNumber=part+1,
                UploadId=mp['UploadId'],
            )
            parts.append({'PartNumber': part+1, 'ETag': temp['ETag']})
        s3_dest.complete_multipart_upload(
            Bucket=DEST_BUCKET, Key=DEST_PREFIX + cur,
            UploadId=mp['UploadId'],
            MultipartUpload={'Parts': parts}
        )
    break

In [3]:
a = (file_content).split(b'</html>')

In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [46]:

import re
def extract_url(x):
    try:
        return re.search(b"WARC-Target-URI: (.*)\\r\\n", x, re.MULTILINE).groups()[0]
    except:
        return None

def extract_ssl_or_not(x):
    try:
        return urlparse(x).scheme
    except:
        return None

def extract_contact_form(x):
    return None

def extract_language_country(x):
    try:
        return re.search(b"Accept-Language: (.*)\\r\\n", x, re.MULTILINE).groups()[0].split(b';')[0]
    except:
        return None
    
def extract_language_country_html(x):
    try:
        return x.find('title').get('lang')
    except:
        return None
    
def extract_title(x):
    try:
        return x.find('title').text
    except:
        return None
def extract_generator(x):
    try:
        meta = x.findAll('meta')
        return list(filter(lambda x: x.get('name') == 'generator', meta))[0].get('content')
    except:
        return None
def extract_contact(x):
    try:
        aa = x.findAll('a')
        return list(filter(lambda x: x.text.lower().find('contact') >= 0, aa))[0].get('href')
    except:
        return None
# def extract_meta(x):
#     try:
#         return BeautifulSoup(x[re.search(b"<html", x, re.MULTILINE).start():] + b'</html>').findAll('meta')
#     except:
#         return None
    
def extract_language_country(x):
    try:
        b = re.search(b"Accept-Language: (.*)\\r\\n", x, re.MULTILINE).groups()[0]
        b = b.split(b',')
        b = map(lambda x: x.split(b';'), b)
        for a in b:
            item = a[0].split(b'-')
            if len(item) == 2:
                return item[0] , item[1]
            if len(item) == 1:
                return item[0], None
        return None, None
    except Exception as e:
        return None, None
email_tester = re.compile(b"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+")
def extract_all_emails(x):
    try:
        return b'|'.join(email_tester.findall(x))
    except:
        return None

In [49]:
extract_all_emails(a[0])

b'info@commoncrawl.org|info@commoncrawl.org|info@commoncrawl.org'

In [47]:
import gzip
path = '/Users/z.abidin/CC-NEWS-20220228233520-00697.warc.gz'
with gzip.open(path, 'rb') as f:
    file_content = f.read()


docs = []
for idx, x in enumerate(a[:20]):
    try:
        html = BeautifulSoup(x[re.search(b"<html", x, re.MULTILINE).start():] + b'</html>')
    except Exception as e:
        html = None
    
    url = extract_url(x)
    language, country = extract_language_country(x)
    docs.append({
        'url': url,
        'http_s': extract_ssl_or_not(url),
        'contact': extract_contact(html),
        'emails': extract_all_emails(x),
        'language': language,
        'country': country,
        'type_of_website': extract_title(html),
        'cms_of_website': extract_generator(html),
    })


In [50]:
df = pd.DataFrame(docs)

In [51]:
df.to_csv('crawler_data_no_header.csv', header=False)

In [34]:
import pandas as pd
df = pd.DataFrame(docs)

for c in df.columns:
    df[c] = df[c].astype('str')
# .to_parquet('crawler.csv')

In [40]:
df.to_json('crawler_data.json', orient='records')

In [123]:
pd.DataFrame(docs).to_parquet('output.parquet')

ArrowTypeError: ('Did not pass numpy.dtype object', 'Conversion failed for column None with type int64')

In [115]:



# for line in a[0].split(b'\r\n')
#     line.startswith('WARC-Target-URI')a
# 1 WARC-Target-URI
# 2 WARC-Target-URI
# 3
# 4 Accept-Language: en-US
# 5 regex email
# 6 Accept-ht: en-US
# 7 <title>
# 8 regex "wordpress", "wix.com", "joomla", 'X\-Generator\: Drupal'


In [72]:
str(a[0].decode('utf-8'))

'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2022-02-28T23:35:20Z\r\nWARC-Filename: CC-NEWS-20220228233520-00697.warc.gz\r\nWARC-Record-ID: <urn:uuid:18b6ebd5-2a4e-4b8b-adac-50829b0b57c5>\r\nContent-Type: application/warc-fields\r\nContent-Length: 442\r\n\r\nsoftware:: StormCrawler 1.17 https://stormcrawler.net/\r\ndescription: News crawl for Common Crawl\r\nhttp-header-user-agent: CCBot/3.0 (https://commoncrawl.org/faq/; info@commoncrawl.org)\r\nhttp-header-from: info@commoncrawl.org\r\nrobots: checked by crawler-commons 1.1 (https://github.com/crawler-commons/crawler-commons)\r\nformat: WARC File Format 1.1\r\nconformsTo: https://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: request\r\nWARC-IP-Address: 184.25.127.152\r\nWARC-Record-ID: <urn:uuid:bb8fb638-cbaf-4a9e-a7e1-ae7854f48008>\r\nContent-Length: 416\r\nWARC-Date: 2022-02-28T23:35:20Z\r\nWARC-Target-URI: https://www.hindustantimes.com/cities/gurugram-news/after-a-two-

In [47]:
str(a[0].decode('utf-8'))

'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2022-02-28T23:35:20Z\r\nWARC-Filename: CC-NEWS-20220228233520-00697.warc.gz\r\nWARC-Record-ID: <urn:uuid:18b6ebd5-2a4e-4b8b-adac-50829b0b57c5>\r\nContent-Type: application/warc-fields\r\nContent-Length: 442\r\n\r\nsoftware:: StormCrawler 1.17 https://stormcrawler.net/\r\ndescription: News crawl for Common Crawl\r\nhttp-header-user-agent: CCBot/3.0 (https://commoncrawl.org/faq/; info@commoncrawl.org)\r\nhttp-header-from: info@commoncrawl.org\r\nrobots: checked by crawler-commons 1.1 (https://github.com/crawler-commons/crawler-commons)\r\nformat: WARC File Format 1.1\r\nconformsTo: https://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: request\r\nWARC-IP-Address: 184.25.127.152\r\nWARC-Record-ID: <urn:uuid:bb8fb638-cbaf-4a9e-a7e1-ae7854f48008>\r\nContent-Length: 416\r\nWARC-Date: 2022-02-28T23:35:20Z\r\nWARC-Target-URI: https://www.hindustantimes.com/cities/gurugram-news/after-a-two-